In [4]:
import pandas as pd
import json

# Constants
AI_ID = '674a85981e5b976f581e24e7'
JSON_FILE = '../data/test.games.json'

def load_json(file_path):
    """Load JSON data from a file."""
    with open(file_path, 'r') as f:
        return json.load(f)

def extract_game_info(game):
    """Extract relevant information from a single game."""
    game_id = game['_id']['$oid']
    player1_id = game['player1']['$oid']
    player2_id = game['player2']['$oid']
    player1_role = game.get('player1Role', '').lower()
    player2_role = game.get('player2Role', '').lower()
    is_ai_game = game.get('isAIGame', False)
    
    # Determine estimator and coin player
    if player1_role == 'estimator':
        estimator_id, estimator_role = player1_id, player1_role
        coin_player_id, coin_player_role = player2_id, player2_role
    elif player2_role == 'estimator':
        estimator_id, estimator_role = player2_id, player2_role
        coin_player_id, coin_player_role = player1_id, player1_role
    else:
        print(f"Game {game_id} does not have a clear estimator. Skipping.")
        return None
    
    return {
        'game_id': game_id,
        'player1_id': player1_id,
        'player2_id': player2_id,
        'player1_role': player1_role,
        'player2_role': player2_role,
        'is_ai_game': is_ai_game,
        'estimator_id': estimator_id,
        'estimator_role': estimator_role,
        'coin_player_id': coin_player_id,
        'coin_player_role': coin_player_role,
        'match_winners': {mw['match']: mw['winner']['$oid'] for mw in game.get('matchWinners', [])},
        'coin_selections': game.get('coinSelections', []),
        'guesses': game.get('guesses', [])
    }

def organize_by_match_round(items, key_fields):
    """Organize items by match and round."""
    organized = {}
    for item in items:
        match_num = item[key_fields['match']]
        round_num = item[key_fields['round']]
        organized.setdefault(match_num, {})[round_num] = item[key_fields['value']]
    return organized

def process_games(data):
    """Process all games and extract match-level data."""
    match_data = []
    adjustment_count = 0
    
    for game in data:
        info = extract_game_info(game)
        if not info:
            continue
        
        coins_by_match_round = organize_by_match_round(
            info['coin_selections'],
            {'match': 'match', 'round': 'round', 'value': 'coins'}
        )
        guesses_by_match_round = organize_by_match_round(
            info['guesses'],
            {'match': 'match', 'round': 'round', 'value': 'guess'}
        )
        
        match_numbers = set(coins_by_match_round.keys()).union(guesses_by_match_round.keys())
        
        for match_num in match_numbers:
            match_id = f"{info['game_id']}_{match_num}"
            coin_selection = [
                coins_by_match_round.get(match_num, {}).get(r) 
                for r in sorted(coins_by_match_round.get(match_num, {}).keys())
            ]
            guess = [
                guesses_by_match_round.get(match_num, {}).get(r) 
                for r in sorted(guesses_by_match_round.get(match_num, {}).keys())
            ]
            original_winner_id = info['match_winners'].get(match_num)
            
            # Determine if estimator won
            estimator_won = any(
                coin == guessed
                for coin, guessed in zip(
                    coins_by_match_round.get(match_num, {}).values(),
                    guesses_by_match_round.get(match_num, {}).values()
                )
                if coin is not None and guessed is not None
            )
            
            calculated_winner_id = info['estimator_id'] if estimator_won else info['coin_player_id']
            adjusted = False
            winner_id = calculated_winner_id
            
            if original_winner_id != calculated_winner_id:
                adjustment_count += 1
                adjusted = True
            else:
                winner_id = original_winner_id
            
            winner_type = 'estimator' if winner_id == info['estimator_id'] else 'coin-player'
            ai_won = (calculated_winner_id == AI_ID)
            
            match_dict = {
                'match_id': match_id,
                'player1_id': info['player1_id'],
                'player2_id': info['player2_id'],
                'player1_role': info['player1_role'],
                'player2_role': info['player2_role'],
                'is_ai_game': info['is_ai_game'],
                'original_winner_id': original_winner_id,
                'calculated_winner_id': calculated_winner_id,
                'winner_id': winner_id,
                'WinnerType': winner_type,
                'AI_won': ai_won,
                'coin_selection': coin_selection,
                'guesses': guess,
                'adjusted': adjusted
            }
            
            match_data.append(match_dict)
    
    # print(f"Number of adjustments made to 'winner_id': {adjustment_count}")
    return pd.DataFrame(match_data)

def identify_ai(row):
    """Identify the AI's role and ID in the match."""
    if row['is_ai_game']:
        if row['player1_id'] == AI_ID:
            return {'role': row['player1_role'], 'id': row['player1_id']}
        elif row['player2_id'] == AI_ID:
            return {'role': row['player2_role'], 'id': row['player2_id']}
    return None

def ai_picked_4_or_5(row):
    """Check if the AI picked or guessed 4 or 5 in the first move."""
    ai_info = row['AI_info']
    if not ai_info:
        return False
    
    role = ai_info['role']
    if role == 'coin-player' and row['coin_selection']:
        return row['coin_selection'][0] in [4, 5]
    elif role == 'estimator' and row['guesses']:
        return row['guesses'][0] in [4, 5]
    return False

def ai_estimator_invalid_guess(row):
    """Check if the AI as estimator made an invalid guess in round 2 or 3."""
    ai_info = row['AI_info']
    if not ai_info or ai_info['role'] != 'estimator':
        return False
    
    guesses = row['guesses']
    coin_selections = row['coin_selection']
    
    if len(guesses) < 3 or len(coin_selections) < 2:
        return False
    
    # Round 2 conditions
    guess_r2 = guesses[1]
    coin_r1 = coin_selections[0]
    condition_r2 = (
        guess_r2 != 0 and
        coin_r1 != 0 and
        guess_r2 <= coin_r1 and
        not (coin_r1 == 5 and guess_r2 == 5)
    )
    
    # Round 3 conditions
    guess_r3 = guesses[2]
    coin_r2 = coin_selections[1]
    condition_r3 = (
        guess_r3 != 0 and
        coin_r2 != 0 and
        guess_r3 <= coin_r2 and
        not ((coin_selections[0] != 5 and coin_selections[1] != 5) and guess_r3 == 5)
    )
    
    return condition_r2 or condition_r3

def filter_ai_games(df):
    """Filter out AI games based on specific conditions."""
    # Identify AI information
    df['AI_info'] = df.apply(identify_ai, axis=1)
    
    # Filter AI games
    ai_games = df[df['AI_info'].notnull()].copy()
    
    if ai_games.empty:
        print("No AI games to process.")
        return df.drop(columns=['AI_info']), pd.DataFrame()
    
    # Apply conditions
    condition1 = ai_games.apply(ai_picked_4_or_5, axis=1)
    condition2 = ai_games.apply(ai_estimator_invalid_guess, axis=1)
    games_to_delete = condition1 | condition2
    
    # print(f"Number of matches to delete based on conditions: {games_to_delete.sum()}")
    
    #if games_to_delete.any():
        #print("\nExamples of deleted matches:")
        #display(ai_games[games_to_delete])
    
    # Exclude deleted matches
    filtered_ai_games = ai_games[~games_to_delete]
    non_ai_games = df[df['AI_info'].isnull()]
    final_df = pd.concat([non_ai_games, filtered_ai_games], ignore_index=True)
    
    return final_df.drop(columns=['AI_info']), ai_games[games_to_delete]

def check_additional_ai_conditions(df):
    """Check additional conditions where AI picked or guessed 4 or 5."""
    df["AI_selected_or_guessed_4_or_5"] = df.apply(
        lambda row: (
            row.get("AI_info") and
            row["AI_info"]["role"] in ["coin-player", "estimator"] and
            (
                (row["AI_info"]["role"] == "coin-player" and row["coin_selection"] and row["coin_selection"][0] in [4, 5]) or
                (row["AI_info"]["role"] == "estimator" and row["guesses"] and row["guesses"][0] in [4, 5])
            )
        ),
        axis=1
    )
    df["AI_selected_or_guessed_4_or_5"].fillna(False, inplace=True)
    return df

def calculate_win_rates(final_df):
    """Calculate win rates for different game types."""
    # Human vs Human
    human_human = final_df[final_df['is_ai_game'] == False]
    total_hh = len(human_human)
    coin_player_wins_hh = ((human_human['winner_id'] == human_human['player1_id']) & (human_human['player1_role'] == 'coin-player')).sum() + \
                          ((human_human['winner_id'] == human_human['player2_id']) & (human_human['player2_role'] == 'coin-player')).sum()
    estimator_wins_hh = ((human_human['winner_id'] == human_human['player1_id']) & (human_human['player1_role'] == 'estimator')).sum() + \
                        ((human_human['winner_id'] == human_human['player2_id']) & (human_human['player2_role'] == 'estimator')).sum()
    coin_player_win_rate_hh = coin_player_wins_hh / total_hh if total_hh > 0 else 0
    estimator_win_rate_hh = estimator_wins_hh / total_hh if total_hh > 0 else 0
    
    # Human vs AI
    human_ai = final_df[final_df['is_ai_game'] == True]
    total_ai = len(human_ai)
    human_coin_wins_ai = ((human_ai['winner_id'] == human_ai['player1_id']) & (human_ai['player1_id'] != AI_ID) & (human_ai['player1_role'] == 'coin-player')).sum() + \
                         ((human_ai['winner_id'] == human_ai['player2_id']) & (human_ai['player2_id'] != AI_ID) & (human_ai['player2_role'] == 'coin-player')).sum()
    human_estimator_wins_ai = ((human_ai['winner_id'] == human_ai['player1_id']) & (human_ai['player1_id'] != AI_ID) & (human_ai['player1_role'] == 'estimator')).sum() + \
                               ((human_ai['winner_id'] == human_ai['player2_id']) & (human_ai['player2_id'] != AI_ID) & (human_ai['player2_role'] == 'estimator')).sum()
    ai_coin_wins_ai = ((human_ai['winner_id'] == AI_ID) & (human_ai['player1_id'] == AI_ID) & (human_ai['player1_role'] == 'coin-player')).sum() + \
                      ((human_ai['winner_id'] == AI_ID) & (human_ai['player2_id'] == AI_ID) & (human_ai['player2_role'] == 'coin-player')).sum()
    ai_estimator_wins_ai = ((human_ai['winner_id'] == AI_ID) & (human_ai['player1_id'] == AI_ID) & (human_ai['player1_role'] == 'estimator')).sum() + \
                           ((human_ai['winner_id'] == AI_ID) & (human_ai['player2_id'] == AI_ID) & (human_ai['player2_role'] == 'estimator')).sum()
    human_coin_win_rate_ai = human_coin_wins_ai / total_ai if total_ai > 0 else 0
    human_estimator_win_rate_ai = human_estimator_wins_ai / total_ai if total_ai > 0 else 0
    
    # Summary Tables
    summary_overall = pd.DataFrame({
        'Spieler': ['Münzspieler', 'Schätzer', 'Gesamt'],
        'Mensch': [coin_player_wins_hh + human_coin_wins_ai,
                   estimator_wins_hh + human_estimator_wins_ai,
                   (coin_player_wins_hh + human_coin_wins_ai) + (estimator_wins_hh + human_estimator_wins_ai)],
        'KI': [ai_coin_wins_ai,
               ai_estimator_wins_ai,
               ai_coin_wins_ai + ai_estimator_wins_ai],
        'Gesamt': [coin_player_wins_hh + human_coin_wins_ai,
                   estimator_wins_hh + human_estimator_wins_ai,
                   (coin_player_wins_hh + human_coin_wins_ai) + (estimator_wins_hh + human_estimator_wins_ai)]
    })
    
    summary_human_human = pd.DataFrame({
        'Spieler': ['Münzspieler', 'Schätzer', 'Gesamt'],
        'Mensch': [coin_player_wins_hh,
                   estimator_wins_hh,
                   coin_player_wins_hh + estimator_wins_hh],
        'Gesamt': [coin_player_wins_hh,
                   estimator_wins_hh,
                   coin_player_wins_hh + estimator_wins_hh]
    })
    
    summary_human_ai = pd.DataFrame({
        'Spieler': ['Münzspieler', 'Schätzer', 'Gesamt'],
        'Mensch': [human_coin_wins_ai,
                   human_estimator_wins_ai,
                   human_coin_wins_ai + human_estimator_wins_ai],
        'KI': [ai_coin_wins_ai,
               ai_estimator_wins_ai,
               ai_coin_wins_ai + ai_estimator_wins_ai],
        'Gesamt': [human_coin_wins_ai + ai_coin_wins_ai,
                   human_estimator_wins_ai + ai_estimator_wins_ai,
                   (human_coin_wins_ai + ai_coin_wins_ai) + (human_estimator_wins_ai + ai_estimator_wins_ai)]
    })
    
    return summary_overall, summary_human_human, summary_human_ai

def calculate_played_coins(final_df):
    """Calculate played coins in round 1 for humans and AI."""
    # Initialize counters
    coins_range = range(6)
    played_coins = {
        'human_estimator': {i: 0 for i in coins_range},
        'human_coin_player': {i: 0 for i in coins_range},
        'ai_estimator': {i: 0 for i in coins_range},
        'ai_coin_player': {i: 0 for i in coins_range},
    }
    
    # Process Human vs Human
    human_human = final_df[final_df['is_ai_game'] == False]
    for _, row in human_human.iterrows():
        for player in ['player1', 'player2']:
            role = row[f'{player}_role']
            if row['coin_selection']:
                coin = row['coin_selection'][0]
                if role == 'estimator':
                    played_coins['human_estimator'][coin] += 1
                elif role == 'coin-player':
                    played_coins['human_coin_player'][coin] += 1
    
    # Process Human vs AI
    human_ai = final_df[final_df['is_ai_game'] == True]
    for _, row in human_ai.iterrows():
        ai_info = identify_ai(row)
        if not ai_info:
            continue
        role = ai_info['role']
        if role == 'estimator' and row['guesses']:
            coin = row['guesses'][0]
            played_coins['ai_estimator'][coin] += 1
        elif role == 'coin-player' and row['coin_selection']:
            coin = row['coin_selection'][0]
            played_coins['ai_coin_player'][coin] += 1
        
        # Identify human player
        human_player = 'player1' if row['player2_id'] == AI_ID else 'player2'
        human_role = row[f'{human_player}_role']
        if row['coin_selection']:
            coin = row['coin_selection'][0]
            if human_role == 'estimator':
                played_coins['human_estimator'][coin] += 1
            elif human_role == 'coin-player':
                played_coins['human_coin_player'][coin] += 1
    
    # Create DataFrames
    df_estimator_human = pd.DataFrame(list(played_coins['human_estimator'].items()), columns=['Coin', 'Played_Count_Estimator_Human'])
    df_coin_player_human = pd.DataFrame(list(played_coins['human_coin_player'].items()), columns=['Coin', 'Played_Count_CoinPlayer_Human'])
    df_estimator_ai = pd.DataFrame(list(played_coins['ai_estimator'].items()), columns=['Coin', 'Played_Count_Estimator_AI'])
    df_coin_player_ai = pd.DataFrame(list(played_coins['ai_coin_player'].items()), columns=['Coin', 'Played_Count_CoinPlayer_AI'])
    
    # Merge DataFrames
    df_played_coins_human = pd.merge(df_estimator_human, df_coin_player_human, on='Coin')
    df_played_coins_ai = pd.merge(df_estimator_ai, df_coin_player_ai, on='Coin')
    
    # Calculate probabilities
    df_played_coins_human['Probability_Estimator_Human'] = df_played_coins_human['Played_Count_Estimator_Human'] / df_played_coins_human['Played_Count_Estimator_Human'].sum()
    df_played_coins_human['Probability_CoinPlayer_Human'] = df_played_coins_human['Played_Count_CoinPlayer_Human'] / df_played_coins_human['Played_Count_CoinPlayer_Human'].sum()
    
    df_played_coins_ai['Probability_Estimator_AI'] = df_played_coins_ai['Played_Count_Estimator_AI'] / df_played_coins_ai['Played_Count_Estimator_AI'].sum()
    df_played_coins_ai['Probability_CoinPlayer_AI'] = df_played_coins_ai['Played_Count_CoinPlayer_AI'] / df_played_coins_ai['Played_Count_CoinPlayer_AI'].sum()
    
    return df_played_coins_human, df_played_coins_ai

def main():
    # Load data
    data = load_json(JSON_FILE)
    
    # Process games into DataFrame
    filtered_games = process_games(data)
    display(filtered_games)
    
    # Filter AI games based on conditions
    final_filtered_games, deleted_matches = filter_ai_games(filtered_games)
    
    # Additional AI condition checks
    final_filtered_games = check_additional_ai_conditions(final_filtered_games)
    
    # Calculate win rates
    summary_overall, summary_human_human, summary_human_ai = calculate_win_rates(final_filtered_games)
    
    # Display summary tables
    print("\nSpiele Gewonnen (Gesamtsumme):")
    display(summary_overall)
    
    print("\nSpiele Gewonnen Mensch vs Mensch:")
    display(summary_human_human)
    
    print("\nSpiele Gewonnen Mensch vs KI:")
    display(summary_human_ai)
    
    # Calculate played coins probabilities
    df_played_coins_human, df_played_coins_ai = calculate_played_coins(final_filtered_games)
    
    # Display played coins tables
    print("\nGespielte Münzen in Runde 1 für Menschen (Aggregiert):")
    display(df_played_coins_human)
    
    print("\nGespielte Münzen in Runde 1 für die KI:")
    display(df_played_coins_ai)

# Execute the main function
if __name__ == "__main__":
    main()


,match_id,player1_id,player2_id,player1_role,player2_role,is_ai_game,original_winner_id,calculated_winner_id,winner_id,WinnerType,AI_won,coin_selection,guesses,adjusted
0,6722a50c0423dbf2c9b01fed_1,6722a4ff0423dbf2c9b01fd3,6722a5050423dbf2c9b01fde,coin-player,estimator,False,6722a5050423dbf2c9b01fde,6722a5050423dbf2c9b01fde,6722a5050423dbf2c9b01fde,estimator,False,"[1, 3, 4]","[1, 5, 5]",False
1,6746232ecbfe73fa1391572c_1,67460b3644f709bce8090770,674622f1cbfe73fa139156de,coin-player,estimator,False,674622f1cbfe73fa139156de,67460b3644f709bce8090770,67460b3644f709bce8090770,coin-player,False,"[0, 1, 3]","[2, 4, 4]",True
2,67462363cbfe73fa139157c1_1,67460b3644f709bce8090770,674622f1cbfe73fa139156de,coin-player,estimator,False,674622f1cbfe73fa139156de,674622f1cbfe73fa139156de,674622f1cbfe73fa139156de,estimator,False,"[1, 4, 0]","[1, 2, 5]",False
3,67462393cbfe73fa1391585d_1,67460b3644f709bce8090770,674622f1cbfe73fa139156de,coin-player,estimator,False,674622f1cbfe73fa139156de,674622f1cbfe73fa139156de,674622f1cbfe73fa139156de,estimator,False,"[1, 3, 4]","[1, 3, 5]",False
4,67462393cbfe73fa1391585d_2,67460b3644f709bce8090770,674622f1cbfe73fa139156de,coin-player,estimator,False,674622f1cbfe73fa139156de,674622f1cbfe73fa139156de,674622f1cbfe73fa139156de,estimator,False,"[1, 0, 4]","[2, 0, 3]",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,674cd839309c1a915200c9f8_5,674cb20f309c1a9152fd8aae,674a85981e5b976f581e24e7,estimator,coin-player,True,674cb20f309c1a9152fd8aae,674a85981e5b976f581e24e7,674a85981e5b976f581e24e7,coin-player,True,"[0, 3, 5]","[1, 0, 2]",True
569,674cd862309c1a915200f048_1,674cb20f309c1a9152fd8aae,674a85981e5b976f581e24e7,estimator,coin-player,True,674a85981e5b976f581e24e7,674cb20f309c1a9152fd8aae,674cb20f309c1a9152fd8aae,estimator,False,"[1, 2]","[0, 2]",True
570,674cd862309c1a915200f048_2,674cb20f309c1a9152fd8aae,674a85981e5b976f581e24e7,estimator,coin-player,True,674cb20f309c1a9152fd8aae,674cb20f309c1a9152fd8aae,674cb20f309c1a9152fd8aae,estimator,False,[1],[1],False
571,674cd862309c1a915200f048_3,674cb20f309c1a9152fd8aae,674a85981e5b976f581e24e7,estimator,coin-player,True,674cb20f309c1a9152fd8aae,674a85981e5b976f581e24e7,674a85981e5b976f581e24e7,coin-player,True,"[3, 5, 5]","[1, 0, 2]",True



Spiele Gewonnen (Gesamtsumme):


,Spieler,Mensch,KI,Gesamt
0,Münzspieler,107,77,107
1,Schätzer,249,87,249
2,Gesamt,356,164,356



Spiele Gewonnen Mensch vs Mensch:


,Spieler,Mensch,Gesamt
0,Münzspieler,76,76
1,Schätzer,137,137
2,Gesamt,213,213


...

,Spieler,Mensch,KI,Gesamt
0,Münzspieler,31,77,108
1,Schätzer,112,87,199
2,Gesamt,143,164,307



Gespielte Münzen in Runde 1 für Menschen (Aggregiert):


,Coin,Played_Count_Estimator_Human,Played_Count_CoinPlayer_Human,Probability_Estimator_Human,Probability_CoinPlayer_Human
0,0,97,86,0.244949,0.265432
1,1,116,91,0.292929,0.280864
2,2,96,70,0.242424,0.216049
3,3,70,59,0.176768,0.182099
4,4,11,12,0.027778,0.037037
5,5,6,6,0.015152,0.018519



Gespielte Münzen in Runde 1 für die KI:


,Coin,Played_Count_Estimator_AI,Played_Count_CoinPlayer_AI,Probability_Estimator_AI,Probability_CoinPlayer_AI
0,0,12,48,0.102564,0.253968
1,1,47,56,0.401709,0.296296
2,2,34,48,0.290598,0.253968
3,3,24,37,0.205128,0.195767
4,4,0,0,0.000000,0.000000
5,5,0,0,0.000000,0.000000
